In [1]:
# i detta exempel kodar vi en enkel linjärklassificerare 

import torch.nn as nn  #ger oss tillgång till layers 
import torch.nn.functional as F # ger oss aktiveringsfunktioner(sigmoid, RELU)
import torch.optim as optim # optimeringsfunktioner
import torch as T # vårt baspaket 



In [2]:
class LinearClassifier(nn.Module):
    ''' Vi deriverar denna klass från nn-modulen eftersom vi vill 
    ha tillgång till dess parametrar. Konstruktorn ska kunna ta 
    learning rate, antalet klasser och dimensionen på inputen'''
    def __init__(self, lr, n_classes, input_dims):
        super(LinearClassifier, self).__init__() 
        # alternativt ekvivalent till super().__init__() 
        
        # nästa steg vill vi deklarera våra layers för nätverket
        self.fc1 = nn.Linear(*input_dims, 128) 
        self.fc2 = nn.Linear(128, 256) # första ger ut 128 units vilket innebär att fc2 måste ha ingångsvärde 128 
        self.fc3 = nn.Linear(256, n_classes)
        
        # optimeraren är Adam, som är en stokastisk gradient descent, 
        # med momentum som är en adaptiv "learningrate" algoritm 
        self.optimizer = optim.Adam(self.parameters(), lr=lr) # self.parameters kommer från nn.module 
        
        #Nästa steg är att deklarera en "förlustfunktion"
        
        self.loss = nn.CrossEntropyLoss() # nn.MSELoss()
        
        '''om vi nu har en GPU vi använder så vill ha ett sätt att skicka 
        hela skiten(nätverket) till den.'''
        
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device) #skickar hela nätverket till GPUn
        
    
    
    def forward(self, data): 
        '''sköter forward-proppen för oss. I vårt fall kommer vi skicka datan till 
        det första lagret och aktivera den, sedan vidare till den andra osv.'''
        
        layer1 = F.sigmoid(self.fc1(data))
        layer2 = F.sigmoid(self.fc2(layer1)) #datan kmr nu givetvs från första skiktet
        layer3 = self.fc3(layer2) # behöver ingen aktivering från sigmoid då Entropifunken sköter det
        
        return layer3 
    
    def learn(self, data, labels):
        '''......'''
        self.optimizer.zero_grad() #För att "nolla" alla gradients mellan iterationerna. 
        data = T.tensor(data).to(self.device) #Konverting av datan till tensorer 
        labels = T.tensor(labels).to(self.device)
        
        
        predictions = self.forward(data)
        
        cost = self.loss(predictions, labels)
        
        cost.backward()
        self.optimizer.step()
        
        
        
        